In [10]:
using DualNumbers, LaTeXStrings, Plots
pyplot()

Plots.PyPlotBackend()

In [11]:
"""
    ciclosestables!(xx, f, nit, nout, cc)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0,cc)` tenga sentido. La idea es los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`. 
"""
function ciclosestables!(xx, f, nit, nout, cc)
    @assert nit > 0 && nout > 0
    
    # Primeros nit iterados
    x0 = 0.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    
    return nothing
end

"""
    diagbifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diagbifurc(f, nit, nout, crange)
    xx = Vector{Float64}(nout)
    #Lo voy a guardar en un array.
    ff = Array{Float64,2}(nout, length(crange))
    #ff = Any[]
    
    for ic in eachindex(crange)
        c = crange[ic]
        ciclosestables!(xx, f, nit, nout, c)
        
        #push!(ff, xx)
        
        ff[:,ic] = xx
        
    end
    
    return ff
end

diagbifurc (generic function with 1 method)

In [12]:
Qc(x,c) = x^2 + c

crange = 0.25:-1/2^10:-1.0

ff = diagbifurc(Qc, 1000, 256, crange); 
cc = ones(size(ff)[1])*crange';

# Esto cambia las matrices en vectores; ayuda un poco para los dibujos
#ff = reshape(ff, size(ff)[1]*size(ff)[2]);
#cc = reshape(cc, size(ff));

In [13]:
figure(figsize=(6,4))
plot(cc, ff, "b,")
plot([-1.2,-1.5,-1.5,-1.2,-1.2],[-1.5,-1.5,-0.9,-0.9,-1.5], "k-")
plot([-1,0.5],[0.0,0.0], "r-")
xlabel(L"c")
ylabel(L"x_\infty")
title("Fig. 1")

LoadError: LoadError: UndefVarError: figure not defined
while loading In[13], in expression starting on line 1

-----

In [14]:
function find_one_bif(ff, cc)

    Resta = []
    bif = 0

    for i in 1:length(ff[1,:])

        resta = abs(ff[1, i] - ff[2, i])

        if resta > 0.0001

            bif = i
            break

        end

    end

    bif, cc[1, bif]

end

find_one_bif (generic function with 1 method)

In [15]:
find_one_bif(f1, c1) #funciona! Aunque está bien chafa.

LoadError: LoadError: UndefVarError: f1 not defined
while loading In[15], in expression starting on line 1

#### voy a tratar de refinar las orbitas usando métdo de newton

Es necesario tener una función que te genera la función iterada en sí misma. Entonces...

### Metaprogramming (cuadrica y más allá...)

In [16]:
nombre(n::Int) = symbol( string("F_", n) )

nombre (generic function with 1 method)

In [131]:
function itera_funcion_identidad(n)
    
    x = "x^2 + c"

    for i in 1:n-1

        x = "($x)^2 +c"

    end

    ex = parse(x)
    ex_ret = :( $(nombre(n))(x, c) = $ex )
    ex_ret
end 

itera_funcion_identidad (generic function with 1 method)

In [132]:
itera_funcion_identidad(1)

:(F_1(x,c) = begin  # In[131], line 12:
            x ^ 2 + c
        end)

Funciona con `DualNumbers` y está muy chido!

### Adecuando `compute_roots`

Lo que busco es darle de comer las orbitas de la función de arriba para que la refine

In [145]:
"""
    compute_roots(f::Function, xx, c)
    Out: roots

xx is an array, cc is the parameter, f is the funtion

"""
function compute_roots(f::Function, A, cc)
    
    roots = similar(A)

    for j in 1:length(A[1,:])
        
        c = cc[1, j]
        
        for i in 1:length(A[:,1])
            
            xi = Dual(A[i, j], 1)

            # 100 iterations of Newton's method
            for k in 1:100
                x_2 = realpart(xi) - (realpart(f(xi, c)) - realpart(xi)) / (dualpart(f(xi, c)) - 1) 
                xi = Dual(x_2, 1)
            end

            roots[i, j] = realpart(xi)
        end
    end
    
    roots
end

compute_roots (generic function with 1 method)

In [19]:
function delete_equals(A)

    for j  in 1:length(A)
        for i  in 1:length(A)

            if i == j
                
                nothing

                elseif abs(abs(A[i]) - abs(A[j])) < 1e-5

                A[i] = NaN
                
            end
        end
    end
    
    deleteat!(A,find(isnan, A))
    
end

delete_equals (generic function with 1 method)

# Ahora integro todo lo que hice arriba al método de Luis

Pasos para acomplar las funciones:
1. `diagbifuc`: se crea una matriz con las orbitas de las iteraciones.
2. `computeroots`: refinamos las orbitas.
    - itera_funcion
    - Condiciones para aplicar la cuádrica.
    
3. Ver si poner los datos en una matriz o en un array.

--------

In [146]:
Qc(x,c) = x^2 + c

crange = 0.25:-1/2^10:-1.0

ff = diagbifurc(Qc, 10000, 1, crange); 
cc = ones(size(ff)[1])*crange';

In [147]:
ff

1x1281 Array{Float64,2}:
 0.4999  0.46875  0.455806  0.445873  0.4375  …  -0.998043  -0.999022  -1.0

In [148]:
eval(itera_funcion_identidad(2))

F_2 (generic function with 1 method)

In [149]:
eval(itera_funcion_identidad(1))

F_1 (generic function with 1 method)

In [150]:
ff_roots = compute_roots(F_2, ff, cc)

1x1281 Array{Float64,2}:
 0.5  0.46875  0.455806  0.445873  0.4375  …  -0.998043  -0.999022  -1.0

In [151]:
compute_roots(F_1, ff, cc)

1x1281 Array{Float64,2}:
 0.5  0.46875  0.455806  0.445873  0.4375  …  -0.61716  -0.617597  -0.618034

> Ya quedó el método de newton listo para usarse. Abajo hago un método de newton que calcula la raíz de una sola condición inicial y no de toda la matriz.

In [133]:
function compute_roots_paso(f::Function, x0, c)
            
            xi = Dual(x0, 1)

            # 1000 iterations of Newton's method
    for i in 1:1000

        x_2 = realpart(xi) - (realpart(f(xi, c)) - realpart(xi)) / (dualpart(f(xi, c)) - 1)
        xi = Dual(x_2, 1)
    end

    realpart(xi)
end

compute_roots_paso (generic function with 1 method)

In [136]:
ff

1x1281 Array{Float64,2}:
 0.4999  0.46875  0.455806  0.445873  0.4375  …  -0.998043  -0.999022  -1.0

In [143]:
for i in 1:length(ff)
    
    ff[i] = compute_roots_paso(F_2, ff[i], crange[i])
    
end

In [144]:
ff

1x1281 Array{Float64,2}:
 0.5  0.46875  0.455806  0.445873  0.4375  …  -0.61716  -0.617597  -0.618034

In [141]:
compute_roots_paso(F_1, ff[1], crange[1])

0.49999999364714787

> El siguiente paso es evaluart el punto en $Q^p_c (x_0) = x_1$ y comparar $x_0$ y $x_i$ usando una toleracia. Si la distancia entre esos dos puntos es mayor a la tolerancia, es una bifurcación. 